## HotChili Analytics trading notebook template
#### Configure by setting ALGO_NAME in cell below.
#### Run various options (backtest, ingest, live) by uncommenting one cell 

In [1]:
%matplotlib inline
%load_ext zipline

# %reload_ext zipline # Uncomment and use this when already loaded zipline extension for magic cell usage.

/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))


In [2]:
import pandas as pd

# Options you can uncomment and set:

# pd.set_option("max_colwidth", 300)
# pd.set_option("display.max_rows", 300)
# pd.set_option("display.max_columns", 50)
# pd.set_option('precision', 2)
# pd.options.display.float_format = '{:20,.2f}'.format

In [3]:
import os

hca_root_path = os.environ['HCA_ROOT']
print(f"hca_root_path = {hca_root_path}")

hca_root_path = /home/hca-ws2004/hca


# Construct algorithm strategy path names

Assumptions:

- the strategy is in a directory with the same name as the strategy in `ALGO_NAME` below
- the strategy is located in the hca-resources directory, which is located relative to `hca_root_path`, found above

In [4]:
ALGO_NAME = "HCA_AllWeatherOptimizeVolatility" # <--- Supply name here

Other variables are derived from `ALGO_NAME`:

In [5]:
HCA_RESOURCES_PATH = hca_root_path + "/hca-resources/" 
ALGO_PATH          = HCA_RESOURCES_PATH + ALGO_NAME + "/" 
ALGO_BT            = ALGO_PATH + ALGO_NAME + ".py"
ALGO_BT_OUT        = ALGO_PATH + ALGO_NAME + ".pkl"
ALGO_LIVE          = ALGO_PATH + ALGO_NAME + "_Live" + ".py"

print(f"""
ALGO_NAME          = {ALGO_NAME}
HCA_RESOURCES_PATH = {HCA_RESOURCES_PATH}
ALGO_PATH          = {ALGO_PATH}
ALGO_BT            = {ALGO_BT}
ALGO_BT_OUT        = {ALGO_BT_OUT}
ALGO_LIVE          = {ALGO_LIVE}

Contents of algo directory:
""")

!ls $ALGO_PATH


ALGO_NAME          = HCA_AllWeatherOptimizeVolatility
HCA_RESOURCES_PATH = /home/hca-ws2004/hca/hca-resources/
ALGO_PATH          = /home/hca-ws2004/hca/hca-resources/HCA_AllWeatherOptimizeVolatility/
ALGO_BT            = /home/hca-ws2004/hca/hca-resources/HCA_AllWeatherOptimizeVolatility/HCA_AllWeatherOptimizeVolatility.py
ALGO_BT_OUT        = /home/hca-ws2004/hca/hca-resources/HCA_AllWeatherOptimizeVolatility/HCA_AllWeatherOptimizeVolatility.pkl
ALGO_LIVE          = /home/hca-ws2004/hca/hca-resources/HCA_AllWeatherOptimizeVolatility/HCA_AllWeatherOptimizeVolatility_Live.py

Contents of algo directory:

HCA_AllWeatherOptimizeVolatility.ipynb
HCA_AllWeatherOptimizeVolatility_Live.py
HCA_AllWeatherOptimizeVolatility.pkl
HCA_AllWeatherOptimizeVolatility.py
HCA_AllWeatherOptimizeVolatility_tearsheet.ipynb


## Zipline backtest: 

- Method: Jupyter magic cell (%%) 
- Execution of zipline code in cell containing command line command
- Uncomment first line and hit (shift-enter) inside the cell to run simulation backtest

In [6]:
# %%zipline --start=2018-1-1 --end=2020-8-10 -b sharadar-eqfd -o $ALGO_BT_OUT

# Source: adapted from various algos on quantopian
# HCA Conversion Date: 09-05-2020
# Conversion Author: Anthony Garner


import matplotlib.pyplot as plt
import numpy as np
import math

from zipline.api import (order, cancel_order, get_open_orders, symbol, symbols, 
                         date_rules, time_rules, order_target_percent, 
                         record, schedule_function, get_datetime)
from trading_calendars import get_calendar


def initialize(context):
    schedule_function(func=trade, date_rule=date_rules.every_day(),
                      time_rule=time_rules.market_open(),half_days=True)
    context.asserts = symbols('SPY','IEF')

    context.rebalance_date = 0
    context.fired = False
    context.rebalance_inteval = 'M'#'Q', #'D', #'M' #'Q' #'Y'

    context.asserts_position = [0.5, 0.5]
    context.volatility_policy = True
    #unused if volatility_policy is false
    context.volatility_days = 252
    context.volatility_price_history = 66
    #set at less than 1 to ensure no leverage
    context.leverage_buffer=0.90


def handle_data(context, data):
    record(SPY=data[symbol('SPY')].price)


def is_new_day(context, now):
    return ( (now.year > context.rebalance_date.year) or (now.month > context.rebalance_date.month) or((now.month == context.rebalance_date.month) and (now.day > context.rebalance_date.day)))             


def is_new_month(context, now):
    return ((now.year > context.rebalance_date.year) or ((now.year == context.rebalance_date.year) and (now.month > context.rebalance_date.month)))


def is_new_quarter(context, now):
    return ((now.year > context.rebalance_date.year) or ((now.year == context.rebalance_date.year) and (now.month == context.rebalance_date.month + 3)))


def is_new_year(context, now):
    return (now.year > context.rebalance_date.year)


def need_rebalance(context, now):
    return ((context.rebalance_inteval == 'Y' and is_new_year(context, now))or 
           (context.rebalance_inteval == 'Q' and is_new_quarter(context, now)) or 
           (context.rebalance_inteval == 'M' and is_new_month(context, now)) or 
           (context.rebalance_inteval == 'D' and is_new_day(context, now)))


# Compute historical volatility  
def compute_volatility(price_history, days):  
    # Compute daily returns  
    daily_returns = price_history.pct_change().dropna().values  
    # Compute daily volatility  
    historical_vol_daily = np.std(daily_returns,axis=0)  
    # Convert daily volatility to annual volatility, assuming 252 trading days  
    historical_vol_annually = historical_vol_daily*math.sqrt(days)  
    # Return estimate of annual volatility  
    return historical_vol_annually


def compute_asserts_volatility(context, data):
    price_history = data.history(context.asserts, "price", context.volatility_price_history, "1d")
    vol = 1.0/(compute_volatility(price_history, context.volatility_days))
    #print("vol: " + str(vol))
    sum = np.sum(vol)
    context.asserts_position = vol / sum
    #print("asserts_position: " + str(context.asserts_position))


def init_portfolio(context, data):
    if context.volatility_policy:
        compute_asserts_volatility(context, data)
    for i in range(0, len(context.asserts)):
        #print("rebalance " + context.asserts[i].symbol + " to:" + str(context.asserts_position[i]*100) + "%")
        order_target_percent(context.asserts[i], context.asserts_position[i]* context.leverage_buffer)    


def rebalance(context, data):
    if context.volatility_policy:
        compute_asserts_volatility(context, data)
    for i in range(0, len(context.asserts)):
        if data.can_trade(context.asserts[i]):
            #print("rebalance " + context.asserts[i].symbol + " to:" + str(context.asserts_position[i]*100) + "%")
            order_target_percent(context.asserts[i], context.asserts_position[i]* context.leverage_buffer)


def trade(context, data):
    if not context.fired:
        context.rebalance_date = get_datetime()
        #print("build portfolio at " + str(context.rebalance_date))
        init_portfolio(context, data)
        context.fired = True
    else:
        now = get_datetime()
        if (need_rebalance(context, now)):
            #print("new rebalance arrivid:" + str(now))
            context.rebalance_date = now
            rebalance(context, data)


def analyze(context, perf):
    ax1 = plt.subplot(211)
    perf.portfolio_value.plot(ax=ax1)
    ax2 = plt.subplot(212, sharex=ax1)
    perf.SPY.plot(ax=ax2)
    plt.gcf().set_size_inches(18, 8)
    plt.show()

## Display your current bundles

In [7]:
#!zipline bundles # Finds all bundles

## Ingest Sharadar funds assets for today

In [8]:
# Ingest Sharadar funds assets for today, if needed.

# Only need to ingest Funds for this algo, and this takes less processing time and system memory than ingesting
# all of Sharadar Equities plus Funds bundle (sharadar-eqfd)
# !zipline ingest -b sharadar-funds

## Zipline backtest, alternative method

- Method: command line
- Execution of zipline code, located in a file, using below command line execution with magic (`!`) invocation
- This line can also be run in a terminal by copying everything past the `!` and pasting (shift-insert) it into the target terminal

In [9]:
!zipline run -f $ALGO_BT  --start=2018-1-1 --end=2021-04-09 -b sharadar-funds -o $ALGO_BT_OUT

/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))
extension: hca_root_path = /home/hca-ws2004/hca
extension:TODAY_STR = 2021-04-10
extension:TWO_YR_AGO_STR = 2019-04-10
extension:  start_date=2019-04-10 end_date = 2021-04-10
[2021-04-11 03:17:04.142386] INFO: Loader: Cache at /home/hca-ws2004/zipline-broker/data/SPY_benchmark.csv does not have data from 1990-01-02 00:00:00+00:00 to 2021-04-09 00:00:00+00:00.

[2021-0

## Run Zipline live on IB-TWS via command line

- Method: command line
- Execution of zipline code using below command line execution using magic (`!`) invocation

**Notes:** 
- IB-TWS or IB-Gateway must be running, with `IB_ACCT` and `IB_URI` port being correct to live trade
- Change `I_WANT_TO_RUN_THIS_CODE` to `True` below to run zipline live on IB-TWS/IB-Gateway

In [10]:
TODAY = pd.datetime.today().strftime("%Y-%m-%d")
print("TODAY = {}".format(TODAY))

TODAY = 2021-04-10


In [11]:
ALGO_STATE = ALGO_PATH + "strategy.state" 
ALGO_RTB   = ALGO_PATH + "realtime-bars/"

# Edit the following URI to match your IB account and port info.
IB_ACCT = "DU1568488"
IB_URI = "127.0.0.1:7497:1301"

# Change following to 'True' and run cell (control-enter) to execute live run.
I_WANT_TO_RUN_THIS_CODE = False
#I_WANT_TO_RUN_THIS_CODE = True

if I_WANT_TO_RUN_THIS_CODE:
    
    !zipline run \
        -s $TODAY \
        -f $ALGO_LIVE \
        --bundle hca-symbol \
        --broker ib \
        --broker-uri $IB_URI \
        --broker-acct $IB_ACCT \
        --data-frequency daily \
        --state-file $ALGO_STATE \
        --realtime-bar-target $ALGO_RTB 